**Denizalp Goktas (dg2906), Da Hua Chen (dc2802)**





*In order to run the model, run all below in order*



# Set Up Environment

In [0]:
!pip install tf-nightly-gpu-2.0-preview==2.0.0-dev20190227 --force-reinstall

  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Read timed out. (read timeout=15)",)': /packages/33/77/d923be0087f53bc864ae65334ab4ee6a97223387f1395a10400c524980bb/tf_nightly_gpu_2.0_preview-2.0.0.dev20190227-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 331.8MB 63kB/s 
    100% |████████████████████████████████| 17.3MB 1.7MB/s 
    100% |████████████████████████████████| 1.2MB 8.3MB/s 
    100% |████████████████████████████████| 61kB 23.3MB/s 
    100% |████████████████████████████████| 102kB 25.9MB/s 
    100% |████████████████████████████████| 61kB 19.7MB/s 
    100% |████████████████████████████████| 61kB 24.8MB/s 
    100% |████████████████████████████████| 358kB 11.4MB/s 
    100% |████████████████████████████████| 10.8MB 2.8MB/s 
    100% |████████████████████████████████| 3.0MB 8.3MB/s 
    100% |████████████

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


In [0]:
import pickle
import numpy as np
import os, sys, time 
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from datetime import datetime
import os

path = "/gdrive/My Drive/Colab Notebooks/DL-HW2/"
os.chdir(path) 
!python -c 'import keras; print(keras.__version__)'
!python3 -c 'import tensorflow as tf; print(tf.__version__)'

Using TensorFlow backend.
2.2.4
2.0.0-dev20190227


# Modified Unet blocks

The models below are a Unet model modified so that they have the ResNet "property" which allows the gradients to not vanish and improves training. We achieve this by putting skip connections in between the convolutional layers.

In [0]:
#Modify Unet with skips to implement resnet

def conv_block(x, n_channels, droprate = 0.25):
    """ for UNet """
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x_short = x
    x = Conv1D(n_channels, 3, padding = 'same', kernel_initializer = 'he_normal')(x)
    x = Dropout(droprate)(x)
    x = BatchNormalization()(x)
    x = Add()([x, x_short])
    x = ReLU()(x)
    x_short = x
    x = Conv1D(n_channels, 3, padding = 'same', kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = Add()([x, x_short])
    return x

def up_block(x, n_channels):
    """ for UNet """
    x_short = x
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = UpSampling1D(size = 2)(x)
    x = Conv1D(n_channels, 2, padding = 'same', kernel_initializer = 'he_normal')(x)
    x = Add()([x, x_short])
    return x

def Conv_UNet(x, droprate=0.25):
    """ 1-D Convolutional UNet https://arxiv.org/abs/1505.04597 """

    conv0 = Conv1D(192, 3, padding = 'same', kernel_initializer = 'he_normal')(x)

    conv1 = conv_block(conv0, 128, droprate)
    pool1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = conv_block(pool1, 192, droprate)
    pool2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = conv_block(pool2, 384, droprate)
    pool3 = MaxPooling1D(pool_size=2)(conv3)

    conv4 = conv_block(pool3, 512, droprate)

    pool4 = MaxPooling1D(pool_size=2)(conv4)
    conv5 = conv_block(pool4, 1024, droprate)
    up5 = conv5

    up4 = up_block(up5, 512)
    up4 = concatenate([conv4,up4], axis = 2)
    up4 = conv_block(up4, 512, droprate)

    up4 = conv4

    up3 = up_block(up4, 384)
    up3 = concatenate([conv3,up3], axis = 2)
    up3 = conv_block(up3, 384, droprate)

    up2 = up_block(up3, 192)
    up2 = concatenate([conv2,up2], axis = 2)
    up2 = conv_block(up2, 192, droprate)

    up1 = up_block(up2, 128)
    up1 = concatenate([conv1,up1], axis = 2)
    up1 = conv_block(up1, 128, droprate)

    up1 = BatchNormalization()(up1)
    up1 = ReLU()(up1)

    return up1


# Unet + RNN model with attention

Note that we have implemented only one model that outputs distances and angles by using the MSA features since it outputted the best result for our purposes. The model below is our submission for all three questions

**The model is composed of the following:**


1.   Embed raw sequences and the q8 sequences, pad all entries so that they are of the same size. 

2.   Concatenate the embedded sequences and MSA informations.

3.  Take the modified Conv_unet of the concatenated input.

4. Multiply each entry of the Conv_unet output by the outputs of the attention mechanism implemented for the Conv_unet.

5. Take the Bidirectional GRU of the initial concatenated input.

6. Multiply each entry of the Bidirectional GRU output by the outputs of the attention mechanism implemented for the GRU.

7. Concatenate the the GRU with attention and Conv_unet layers together.

8. Make the concatenated output in step 7 go through a dense layer with softmax activation

9. Use the output to predict angles and distance matrices.








**Hyperparameters**: 
1. Epochs : 60
2. Piecewise constant decay optimizer with decays steps of [0.0001, 0.00005, 0.00002]  that changed at the 30th and the 48th iterations.

In [0]:
import os, sys, time 
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import *
from model_utils import *
from datetime import datetime

#######################################
# require tensorflow 2.0 
# CPU-only: pip install tf-nightly-2.0-preview
# GPU: pip install tf-nightly-2.0-gpu-preview
print(tf.__version__)
#######################################

# python train_unet.py

model_name = os.path.basename(sys.argv[0]).split(".")[0]   # or model name
model_name = datetime.now().strftime("%Y%m%d-%H%M%S") + "-" + model_name

#######################################
# Load data 
#######################################

# replace this with your data directory
data_dir = "/gdrive/My Drive/Colab Notebooks/DL-HW2/cu-deep-learning-spring19-hw2/"

folds_use = [1,2,3,4,5,6,7,8,9,10]
data_fields = [[] for _ in range(9)]
for i in folds_use:
    with open(os.path.join(data_dir, 'train_fold_{}.pkl'.format(i)), 'rb') as f:
        data = pickle.load(f)
        for j in range(len(data_fields)):
            data_fields[j].append(data[j])

for j in range(len(data_fields)):
    data_fields[j] = np.concatenate(data_fields[j])
indices, pdbs, length_aas, pdb_aas, q8s, dcalphas, psis, phis, msas = data_fields
print("Total number of protein sequences:", len(pdb_aas))

# filter out sequences that are too long 
maxlen_seq = 384
minlen_seq = 10
lenth_mask = (length_aas < maxlen_seq) & (length_aas > minlen_seq)
for j in range(len(data_fields)):
    data_fields[j] = data_fields[j][lenth_mask]
indices, pdbs, length_aas, pdb_aas, q8s, dcalphas, psis, phis, msas = data_fields
msas = [np.stack(x).transpose().astype(np.float32) for x in msas]
phis = [np.array(x) for x in phis]
psis = [np.array(x) for x in psis]
print("Number of protein sequences shorter than {}: {}".format(maxlen_seq, len(pdb_aas)))

#############################################
# Prepare dataset for traininng 
#############################################

# Pad target distance matrix to the same size 
# Mask the padded regions later so that they don't contribute to the loss
dcalphas_pad = np.zeros((len(dcalphas), maxlen_seq, maxlen_seq), dtype=np.float32)
for i in range(len(dcalphas)):
    length = dcalphas[i].shape[0]
    dcalphas_pad[i, :length, :length] = dcalphas[i]

# Pad target torsion angles to the same size 
train_target_phis = np.zeros([len(phis), maxlen_seq], dtype=np.float32)
for i in range(len(phis)):
    train_target_phis[i, :phis[i].shape[0]] = phis[i]
train_target_psis = np.zeros([len(psis), maxlen_seq], dtype=np.float32)
for i in range(len(psis)):
    train_target_psis[i, :psis[i].shape[0]] = psis[i]

# Input data 
train_input_seqs = pdb_aas
train_input_grams = seq2ngrams(train_input_seqs, n=1)
train_q8s = q8s
train_q8s_grams = seq2ngrams(train_q8s, n=1)

# Define tokenizer encoder the input sequence
tokenizer_encoder = text.Tokenizer()
tokenizer_encoder.fit_on_texts(train_input_grams)
tokenizer_encoder_q8s = text.Tokenizer()
tokenizer_encoder_q8s.fit_on_texts(train_q8s_grams)

# Tokenize the input sequences for use in training
train_input_data = tokenizer_encoder.texts_to_sequences(train_input_grams)
train_input_data = sequence.pad_sequences(train_input_data, maxlen = maxlen_seq, padding = 'post', truncating='post')
train_q8s_data = tokenizer_encoder_q8s.texts_to_sequences(train_q8s_grams)
train_q8s_data = sequence.pad_sequences(train_q8s_data, maxlen = maxlen_seq, padding = 'post', truncating='post')

# The number of words and tags to be passed as parameters to the model
n_words = len(tokenizer_encoder.word_index) + 1
n_words_q8s = len(tokenizer_encoder_q8s.word_index) + 1

# extra MSA features 
msa_dim = msas[0].shape[1]
msas_padded = np.zeros([len(msas), maxlen_seq, msa_dim], dtype=np.float32)
for i in range(len(msas)):
    msas_padded[i, :msas[i].shape[0], :] = msas[i]

# Train/validation set split 
training_idx = np.arange(2800)
validation_idx = np.arange(2800, len(pdb_aas))

X_train = train_input_data[training_idx]
X_val = train_input_data[validation_idx]

X_train_q8s = train_q8s_data[training_idx]
X_val_q8s = train_q8s_data[validation_idx]

X_train_msa = msas_padded[training_idx]
X_val_msa = msas_padded[validation_idx]

X_train_seqlen = length_aas[training_idx]
X_val_seqlen = length_aas[validation_idx]

y_train_dist_matrix = dcalphas_pad[training_idx]
y_val_dist_matrix = dcalphas_pad[validation_idx]

y_train_phis = train_target_phis[training_idx]
y_val_phis = train_target_phis[validation_idx]
y_train_psis = train_target_psis[training_idx]
y_val_psis = train_target_psis[validation_idx]

pdb_names_train = pdbs[training_idx]
pdb_names_val = pdbs[validation_idx]


#############################################
# Create a directory to save model checkpoints 
#############################################

if not os.path.exists("logs"):
    os.mkdir("logs")
log_dir = 'logs/{}'.format(model_name)
os.mkdir(log_dir)

with open(os.path.join(log_dir, 'tokenizer_encoder.pickle'), 'wb') as handle:
    pickle.dump(tokenizer_encoder, handle)

with open(os.path.join(log_dir, 'tokenizer_encoder_q8s.pickle'), 'wb') as handle:
    pickle.dump(tokenizer_encoder_q8s, handle)


#############################################
# build a neural network model
#############################################

# Here we use layer functions to build neural network graph (as opposed to defining a Model class)

# Input layer: (None, ) shape allows the network to be flexible to input sequence length 
# inputs: [protein_sequence, q8_sequence, MSA features]
inputs = [Input(shape = (None, )), Input(shape = (None, )), Input(shape = (None, 21))]

# Word (amino acid) embedding layers 
embedding1 = Embedding(input_dim = n_words, output_dim = 128, input_length = None, input_shape=(None,))
embedding2 = Embedding(input_dim = n_words_q8s, output_dim = 64, input_length = None, input_shape=(None,))
embed_input = embedding1(inputs[0]) # raw seq
embed_q8s = embedding2(inputs[1]) # q8


merged_input = concatenate([embed_input, embed_q8s, inputs[2]], axis = 2)

######### Raw Sequence data

#### A seq-to-seq 1-D convolutional U-Net (https://arxiv.org/abs/1505.04597)
# raw_seq = Conv_UNet(embed_input, droprate=0.1)
# q8_seq =  Conv_UNet(embed_q8s, droprate=0.1)
# msa_seq = Conv_UNet(inputs[2], droprate=0.1)
conv1 = Conv_UNet(merged_input, droprate = 0.3)

#### bidirectional RNN
rnn = Bidirectional(GRU(units=32, return_sequences=True, recurrent_dropout=0.2, recurrent_activation='relu', bias_initializer= "ones"))
GRU1 = rnn(merged_input)
GRU1 = BatchNormalization()(conv1)


##################################################
# Attention
##################################################

attention1 = Dense(1, activation='tanh')(GRU1)
attention1 = Flatten()(attention1)
attention1 = Activation('softmax')(attention1)
attention1 = RepeatVector(rnn.output_shape[2]*2)(attention1)
attention1 = Permute([2, 1])(attention1)


attention_merge1 = multiply([attention1, GRU1])



attention2 = Dense(1, activation='tanh')(conv1)
attention2 = Flatten()(attention2)
attention2 = Activation('softmax')(attention2)
attention2 = RepeatVector(conv1.shape[2])(attention2)
attention2 = Permute([2, 1])(attention2)


attention_merge2 = multiply([attention2, conv1])

#### A seq-to-seq 1-D convolutional U-Net (https://arxiv.org/abs/1505.04597)

result = concatenate([attention_merge1, attention_merge2])

#### Torsion angle prediction
# The model output 3 angles (phi, psi, omega) for computing distance matrix. 
# But we only use phi, psi for evaluating angles since omega angle is usually fixed in the protein. 
#y = TimeDistributed(Dense(3, activation = "tanh"))(merged_out)
#angles = tf.multiply(y, np.pi, name="torsion_angles") 

# The RGN paper predict torsion angles using softmax probabilities over a learned alphabet/mixture of angles (https://www.biorxiv.org/content/biorxiv/early/2018/08/29/265231.full-text.pdf)
y = TimeDistributed(Dense(50, activation = "softmax"))(result)
angles = TorsionAngles(alphabet_size=50)(y)

#### Build Model. 
# Define the computational graph using model input and model output
model = tf.keras.Model(inputs, [angles, y])
model.summary()



##################################################
# Train the model
##################################################

batch_size = 32
angle_scale = 180 / np.pi  # convert angles from [-pi, pi] to [-180, 180]
n_iter = int(X_train.shape[0] / batch_size)

#### For RNN (slower)
# n_epochs = 8
# lr_decay_iters = [n_iter * 3.0, n_iter * 6.0]  # lr decay at n_iters * epoch_steps
# lr_steps = [0.001, 0.0005, 0.0001]  # learning rate decay steps

#### For ConvNet
n_epochs = 60
lr_decay_iters = [n_iter * 5.0, n_iter * 8.0]  # lr decay at n_iters * epoch_steps
lr_steps = [0.0001, 0.00005, 0.00002]  # learning rate decay steps

learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(lr_decay_iters, lr_steps)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate_fn)

global_step = tf.Variable(0, trainable=False)
train_loss_history = []
val_loss_history = []

torsion_loss_weight = 1.0 / 50
dist_loss_weight = 1.0 

for epoch in range(1, n_epochs+1):
    
    ################################### 
    # Train
    ################################### 
    idx_shuffle = np.random.permutation(X_train.shape[0])

    train_rmsd_dist = tf.metrics.Mean()
    train_rmsd_dist_norm = tf.metrics.Mean()
    train_rmsd_angle = tf.metrics.Mean()
    train_rmsd_all = tf.metrics.Mean()

    # Iterate through mini-batchs
    for it in range(n_iter):
        # Shuffle (alternatively, use tf.dataset)
        idx_batch = idx_shuffle[it * batch_size : (it+1) * batch_size]
        batch_seq = tf.convert_to_tensor(X_train[idx_batch])
        batch_q8s = tf.convert_to_tensor(X_train_q8s[idx_batch])
        batch_seqlen = tf.convert_to_tensor(X_train_seqlen[idx_batch], dtype=tf.float32)
        batch_dcalphas = tf.convert_to_tensor(y_train_dist_matrix[idx_batch])
        batch_msa = tf.convert_to_tensor(X_train_msa[idx_batch])

        # Compute loss 
        with tf.GradientTape() as tape:
            torsion_angles, y_out = model([batch_seq, batch_q8s, batch_msa])
            phi, psi = torsion_angles[:, :, 0], torsion_angles[:, :, 1]  
            phi_scaled, psi_scaled = phi * angle_scale, psi * angle_scale  # from [-pi, pi] to [-180, 180]
            loss_phi_batch = rmsd_torsion_angle(phi_scaled, y_train_phis[idx_batch], batch_seqlen)
            loss_psi_batch = rmsd_torsion_angle(psi_scaled, y_train_psis[idx_batch], batch_seqlen)
            loss_phi = tf.reduce_mean(loss_phi_batch)  
            loss_psi = tf.reduce_mean(loss_psi_batch)

            dist_matrix, coordinates = DistanceMatrix()(torsion_angles)
            loss_drmsd_batch = drmsd_dist_matrix(dist_matrix, batch_dcalphas, batch_seqlen)
            loss_drmsd = tf.reduce_mean(loss_drmsd_batch)  # drmsd metric
            loss_drmsd_normalized = tf.reduce_mean(loss_drmsd_batch / tf.sqrt(batch_seqlen))  # longer proteins have larger distance
            
            # the optimization objective can be 1. loss_drmsd or 2. (loss_phi + loss_psi), or both
            # optimizing distance mattrix (drmsd) and tortion angles separately probably gives better rerults. 
            # loss_all = loss_drmsd   # distance matrix loss
            # loss_all = loss_phi + loss_psi  # torsion angle loss
            
            loss_all = loss_drmsd * dist_loss_weight + (loss_phi + loss_psi) * torsion_loss_weight
            

        # Compute gradient 
        grads = tape.gradient(loss_all, model.trainable_variables)  # loss includ both distance matrix and torsion angles 
        # grads = tape.gradient(loss_all, model.trainable_variables)
        grads, global_norm = tf.clip_by_global_norm(grads, 5.0)
        
        # Backprop
        optimizer.apply_gradients(zip(grads, model.trainable_variables), global_step)

        # Record metrics
        train_rmsd_dist_norm(loss_drmsd_normalized)
        train_rmsd_dist(loss_drmsd)
        train_rmsd_angle((loss_phi + loss_psi)/2)
        train_rmsd_all(loss_all)

        if it % 10 == 0:
            print("Epoch {:04d} Batch {:03d}/{:03d}: Loss: {:.5g}, dist RMSD: {:.5g}, (length-normalized): {:.5g}, angle RMSD: {:.5g}".format(
                epoch, it, n_iter, loss_all, loss_drmsd, loss_drmsd_normalized, (loss_phi + loss_psi)/2))
        
    
    ###################################  
    # Validation 
    ################################### 
    idx_val = np.arange(X_val.shape[0])
    n_iter_val = int(len(idx_val) / batch_size) + 1  # use all

    val_rmsd_dist = tf.metrics.Mean()
    val_rmsd_dist_norm = tf.metrics.Mean()
    val_rmsd_angle = tf.metrics.Mean()
    val_rmsd_all = tf.metrics.Mean()

    for it in range(n_iter_val):
        idx_batch = idx_val[it * batch_size : (it+1) * batch_size]
        batch_seq = tf.convert_to_tensor(X_val[idx_batch])
        batch_q8s = tf.convert_to_tensor(X_val_q8s[idx_batch])
        batch_seqlen = tf.convert_to_tensor(X_val_seqlen[idx_batch], dtype=tf.float32)
        batch_dcalphas = tf.convert_to_tensor(y_val_dist_matrix[idx_batch])
        batch_msa = tf.convert_to_tensor(X_val_msa[idx_batch])

        with tf.GradientTape() as tape:
            torsion_angles, y_out = model([batch_seq, batch_q8s, batch_msa])
            phi, psi = torsion_angles[:, :, 0], torsion_angles[:, :, 1]
            phi_scaled, psi_scaled = phi * angle_scale, psi * angle_scale  # from [-pi, pi] to [-180, 180]
            loss_phi_batch = rmsd_torsion_angle(phi_scaled, y_val_phis[idx_batch], batch_seqlen)
            loss_psi_batch = rmsd_torsion_angle(psi_scaled, y_val_psis[idx_batch], batch_seqlen)
            loss_phi = tf.reduce_mean(loss_phi_batch)  
            loss_psi = tf.reduce_mean(loss_psi_batch)

            # torsion_angles are on the scale of [-3.14, 3.14], the target labels are on the scale of [-180, 180]. Need to convert after prediction
            dist_matrix, coordinates = DistanceMatrix()(torsion_angles)
            loss_drmsd_batch = drmsd_dist_matrix(dist_matrix, batch_dcalphas, batch_seqlen)
            loss_drmsd = tf.reduce_mean(loss_drmsd_batch)
            loss_drmsd_normalized = tf.reduce_mean(loss_drmsd_batch / tf.sqrt(batch_seqlen))

            #loss_all = loss_drmsd
            # loss_all = loss_phi + loss_psi
            loss_all = loss_drmsd * dist_loss_weight + (loss_phi + loss_psi) * torsion_loss_weight

        # no gradient descent during validation
        val_rmsd_dist_norm(loss_drmsd_normalized)
        val_rmsd_dist(loss_drmsd)
        val_rmsd_angle((loss_phi + loss_psi)/2)
        val_rmsd_all(loss_all)

        if it == 0:
            plot_pred = dist_matrix[:8].numpy()
            plot_gt = batch_dcalphas[:8].numpy()
            plot_names = pdb_names_val[idx_batch][:8]
            plot_len = batch_seqlen[:8].numpy().astype(int)
            plot_rmds = loss_drmsd_batch[:8].numpy()
            plot_dist_matrix(plot_pred, plot_gt, plot_names, plot_len, plot_rmds,
                os.path.join(log_dir, "plot_distance_matrix_epoch-{}.pdf".format(epoch)))

    ################################### 

    # plot history if not using tensorboard
    train_loss_history.append([train_rmsd_all.result(), train_rmsd_dist.result(), train_rmsd_dist_norm.result(), train_rmsd_angle.result()])
    val_loss_history.append([val_rmsd_all.result(), val_rmsd_dist.result(), val_rmsd_dist_norm.result(), val_rmsd_angle.result()])

    print("\n===========================================\n")
    print("Epoch {:03d}: Train Loss: {:.5g}, dist RMSD: {:.5g}, (length-normalized): {:.5g}, angle RMSD: {:.5g} \n ".format(
        epoch, *train_loss_history[-1]) + 
        "             Val Loss: {:.5g}, dist RMSD: {:.5g}, (length-normalized): {:.5g}, angle RMSD: {:.5g}".format(
        *val_loss_history[-1]))
    print("\n===========================================\n")

    # save model checkpoints (this overwrites previous checkpoints)
    tf.saved_model.save(model, log_dir)

    # plot
    train_loss_arr = np.array(train_loss_history).transpose()
    val_loss_arr = np.array(val_loss_history).transpose()
    plot_train_val(train_loss_arr[0], val_loss_arr[0], 
            title="RMSD (dist + angle) loss", savepath=os.path.join(log_dir, "rmsd_all.pdf"))
    plot_train_val(train_loss_arr[1], val_loss_arr[1], 
            title="RMSD (distance matrix) loss", savepath=os.path.join(log_dir, "rmsd_distance_matrix.pdf"))
    plot_train_val(train_loss_arr[2], val_loss_arr[2], 
            title="RMSD (distance matrix, lengh-normalized) loss", savepath=os.path.join(log_dir, "rmsd_distance_matrix_normalized.pdf"))
    plot_train_val(train_loss_arr[3], val_loss_arr[3], 
            title="RMSD (torsion angles) loss", savepath=os.path.join(log_dir, "rmsd_torsion_angles.pdf"))

2.0.0-dev20190227
Total number of protein sequences: 3449
Number of protein sequences shorter than 384: 3155


W0312 22:18:14.363067 140693574170496 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:4080: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 128)    2688        input_7[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 64)     576         input_8[0][0]                    
____________________________________________________________________________________________

In [0]:
import os, sys, time 
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import *
from model_utils import *
from datetime import datetime

#######################################
# require tensorflow 2.0 
# CPU-only: pip install tf-nightly-2.0-preview
# GPU: pip install tf-nightly-2.0-gpu-preview
print(tf.__version__)
#######################################

# python train_unet.py

model_name = os.path.basename(sys.argv[0]).split(".")[0]   # or model name
model_name = datetime.now().strftime("%Y%m%d-%H%M%S") + "-" + model_name

#######################################
# Load data 
#######################################

# replace this with your data directory
data_dir = "/gdrive/My Drive/Colab Notebooks/DL-HW2/cu-deep-learning-spring19-hw2/"

folds_use = [1,2,3,4,5,6,7,8,9,10]
data_fields = [[] for _ in range(9)]
for i in folds_use:
    with open(os.path.join(data_dir, 'train_fold_{}.pkl'.format(i)), 'rb') as f:
        data = pickle.load(f)
        for j in range(len(data_fields)):
            data_fields[j].append(data[j])

for j in range(len(data_fields)):
    data_fields[j] = np.concatenate(data_fields[j])
indices, pdbs, length_aas, pdb_aas, q8s, dcalphas, psis, phis, msas = data_fields
print("Total number of protein sequences:", len(pdb_aas))

# filter out sequences that are too long 
maxlen_seq = 384
minlen_seq = 10
lenth_mask = (length_aas < maxlen_seq) & (length_aas > minlen_seq)
for j in range(len(data_fields)):
    data_fields[j] = data_fields[j][lenth_mask]
indices, pdbs, length_aas, pdb_aas, q8s, dcalphas, psis, phis, msas = data_fields
msas = [np.stack(x).transpose().astype(np.float32) for x in msas]
phis = [np.array(x) for x in phis]
psis = [np.array(x) for x in psis]
print("Number of protein sequences shorter than {}: {}".format(maxlen_seq, len(pdb_aas)))

#############################################
# Prepare dataset for traininng 
#############################################

# Pad target distance matrix to the same size 
# Mask the padded regions later so that they don't contribute to the loss
dcalphas_pad = np.zeros((len(dcalphas), maxlen_seq, maxlen_seq), dtype=np.float32)
for i in range(len(dcalphas)):
    length = dcalphas[i].shape[0]
    dcalphas_pad[i, :length, :length] = dcalphas[i]

# Pad target torsion angles to the same size 
train_target_phis = np.zeros([len(phis), maxlen_seq], dtype=np.float32)
for i in range(len(phis)):
    train_target_phis[i, :phis[i].shape[0]] = phis[i]
train_target_psis = np.zeros([len(psis), maxlen_seq], dtype=np.float32)
for i in range(len(psis)):
    train_target_psis[i, :psis[i].shape[0]] = psis[i]

# Input data 
train_input_seqs = pdb_aas
train_input_grams = seq2ngrams(train_input_seqs, n=1)
train_q8s = q8s
train_q8s_grams = seq2ngrams(train_q8s, n=1)

# Define tokenizer encoder the input sequence
tokenizer_encoder = text.Tokenizer()
tokenizer_encoder.fit_on_texts(train_input_grams)
tokenizer_encoder_q8s = text.Tokenizer()
tokenizer_encoder_q8s.fit_on_texts(train_q8s_grams)

# Tokenize the input sequences for use in training
train_input_data = tokenizer_encoder.texts_to_sequences(train_input_grams)
train_input_data = sequence.pad_sequences(train_input_data, maxlen = maxlen_seq, padding = 'post', truncating='post')
train_q8s_data = tokenizer_encoder_q8s.texts_to_sequences(train_q8s_grams)
train_q8s_data = sequence.pad_sequences(train_q8s_data, maxlen = maxlen_seq, padding = 'post', truncating='post')

# The number of words and tags to be passed as parameters to the model
n_words = len(tokenizer_encoder.word_index) + 1
n_words_q8s = len(tokenizer_encoder_q8s.word_index) + 1

# extra MSA features 
msa_dim = msas[0].shape[1]
msas_padded = np.zeros([len(msas), maxlen_seq, msa_dim], dtype=np.float32)
for i in range(len(msas)):
    msas_padded[i, :msas[i].shape[0], :] = msas[i]

# Train/validation set split 
training_idx = np.arange(2800)
validation_idx = np.arange(2800, len(pdb_aas))

X_train = train_input_data[training_idx]
X_val = train_input_data[validation_idx]

X_train_q8s = train_q8s_data[training_idx]
X_val_q8s = train_q8s_data[validation_idx]

X_train_msa = msas_padded[training_idx]
X_val_msa = msas_padded[validation_idx]

X_train_seqlen = length_aas[training_idx]
X_val_seqlen = length_aas[validation_idx]

y_train_dist_matrix = dcalphas_pad[training_idx]
y_val_dist_matrix = dcalphas_pad[validation_idx]

y_train_phis = train_target_phis[training_idx]
y_val_phis = train_target_phis[validation_idx]
y_train_psis = train_target_psis[training_idx]
y_val_psis = train_target_psis[validation_idx]

pdb_names_train = pdbs[training_idx]
pdb_names_val = pdbs[validation_idx]


#############################################
# Create a directory to save model checkpoints 
#############################################

if not os.path.exists("logs"):
    os.mkdir("logs")
log_dir = 'logs/{}'.format(model_name)
os.mkdir(log_dir)

with open(os.path.join(log_dir, 'tokenizer_encoder.pickle'), 'wb') as handle:
    pickle.dump(tokenizer_encoder, handle)

with open(os.path.join(log_dir, 'tokenizer_encoder_q8s.pickle'), 'wb') as handle:
    pickle.dump(tokenizer_encoder_q8s, handle)


#############################################
# build a neural network model
#############################################

# Here we use layer functions to build neural network graph (as opposed to defining a Model class)

# Input layer: (None, ) shape allows the network to be flexible to input sequence length 
# inputs: [protein_sequence, q8_sequence, MSA features]
inputs = [Input(shape = (None, )), Input(shape = (None, )), Input(shape = (None, 21))]

# Word (amino acid) embedding layers 
embedding1 = Embedding(input_dim = n_words, output_dim = 128, input_length = None, input_shape=(None,))
embedding2 = Embedding(input_dim = n_words_q8s, output_dim = 64, input_length = None, input_shape=(None,))
embed_input = embedding1(inputs[0]) # raw seq
embed_q8s = embedding2(inputs[1]) # q8


merged_input = concatenate([embed_input, embed_q8s, inputs[2]], axis = 2)

######### Raw Sequence data

#### A seq-to-seq 1-D convolutional U-Net (https://arxiv.org/abs/1505.04597)
# raw_seq = Conv_UNet(embed_input, droprate=0.1)
# q8_seq =  Conv_UNet(embed_q8s, droprate=0.1)
# msa_seq = Conv_UNet(inputs[2], droprate=0.1)
conv1 = Conv_UNet(merged_input, droprate = 0.3)

#### bidirectional RNN
rnn = Bidirectional(GRU(units=32, return_sequences=True, recurrent_dropout=0.2, recurrent_activation='relu', bias_initializer= "ones"))
GRU1 = rnn(merged_input)
GRU1 = BatchNormalization()(conv1)


##################################################
# Attention
##################################################

attention1 = Dense(1, activation='tanh')(GRU1)
attention1 = Flatten()(attention1)
attention1 = Activation('softmax')(attention1)
attention1 = RepeatVector(rnn.output_shape[2]*2)(attention1)
attention1 = Permute([2, 1])(attention1)


attention_merge1 = multiply([attention1, GRU1])



attention2 = Dense(1, activation='tanh')(conv1)
attention2 = Flatten()(attention2)
attention2 = Activation('softmax')(attention2)
attention2 = RepeatVector(conv1.shape[2])(attention2)
attention2 = Permute([2, 1])(attention2)


attention_merge2 = multiply([attention2, conv1])

#### A seq-to-seq 1-D convolutional U-Net (https://arxiv.org/abs/1505.04597)

result = concatenate([attention_merge1, attention_merge2])

#### Torsion angle prediction
# The model output 3 angles (phi, psi, omega) for computing distance matrix. 
# But we only use phi, psi for evaluating angles since omega angle is usually fixed in the protein. 
#y = TimeDistributed(Dense(3, activation = "tanh"))(merged_out)
#angles = tf.multiply(y, np.pi, name="torsion_angles") 

# The RGN paper predict torsion angles using softmax probabilities over a learned alphabet/mixture of angles (https://www.biorxiv.org/content/biorxiv/early/2018/08/29/265231.full-text.pdf)
y = TimeDistributed(Dense(50, activation = "softmax"))(result)
angles = TorsionAngles(alphabet_size=50)(y)

#### Build Model. 
# Define the computational graph using model input and model output
model = tf.keras.Model(inputs, [angles, y])
model.summary()



##################################################
# Train the model
##################################################

batch_size = 32
angle_scale = 180 / np.pi  # convert angles from [-pi, pi] to [-180, 180]
n_iter = int(X_train.shape[0] / batch_size)

#### For RNN (slower)
# n_epochs = 8
# lr_decay_iters = [n_iter * 3.0, n_iter * 6.0]  # lr decay at n_iters * epoch_steps
# lr_steps = [0.001, 0.0005, 0.0001]  # learning rate decay steps

#### For ConvNet
n_epochs = 60
lr_decay_iters = [n_iter * 5.0, n_iter * 8.0]  # lr decay at n_iters * epoch_steps
lr_steps = [0.0001, 0.00005, 0.00002]  # learning rate decay steps

learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(lr_decay_iters, lr_steps)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate_fn)

global_step = tf.Variable(0, trainable=False)
train_loss_history = []
val_loss_history = []

torsion_loss_weight = 1.0 / 50
dist_loss_weight = 1.0 

for epoch in range(1, n_epochs+1):
    
    ################################### 
    # Train
    ################################### 
    idx_shuffle = np.random.permutation(X_train.shape[0])

    train_rmsd_dist = tf.metrics.Mean()
    train_rmsd_dist_norm = tf.metrics.Mean()
    train_rmsd_angle = tf.metrics.Mean()
    train_rmsd_all = tf.metrics.Mean()

    # Iterate through mini-batchs
    for it in range(n_iter):
        # Shuffle (alternatively, use tf.dataset)
        idx_batch = idx_shuffle[it * batch_size : (it+1) * batch_size]
        batch_seq = tf.convert_to_tensor(X_train[idx_batch])
        batch_q8s = tf.convert_to_tensor(X_train_q8s[idx_batch])
        batch_seqlen = tf.convert_to_tensor(X_train_seqlen[idx_batch], dtype=tf.float32)
        batch_dcalphas = tf.convert_to_tensor(y_train_dist_matrix[idx_batch])
        batch_msa = tf.convert_to_tensor(X_train_msa[idx_batch])

        # Compute loss 
        with tf.GradientTape() as tape:
            torsion_angles, y_out = model([batch_seq, batch_q8s, batch_msa])
            phi, psi = torsion_angles[:, :, 0], torsion_angles[:, :, 1]  
            phi_scaled, psi_scaled = phi * angle_scale, psi * angle_scale  # from [-pi, pi] to [-180, 180]
            loss_phi_batch = rmsd_torsion_angle(phi_scaled, y_train_phis[idx_batch], batch_seqlen)
            loss_psi_batch = rmsd_torsion_angle(psi_scaled, y_train_psis[idx_batch], batch_seqlen)
            loss_phi = tf.reduce_mean(loss_phi_batch)  
            loss_psi = tf.reduce_mean(loss_psi_batch)

            dist_matrix, coordinates = DistanceMatrix()(torsion_angles)
            loss_drmsd_batch = drmsd_dist_matrix(dist_matrix, batch_dcalphas, batch_seqlen)
            loss_drmsd = tf.reduce_mean(loss_drmsd_batch)  # drmsd metric
            loss_drmsd_normalized = tf.reduce_mean(loss_drmsd_batch / tf.sqrt(batch_seqlen))  # longer proteins have larger distance
            
            # the optimization objective can be 1. loss_drmsd or 2. (loss_phi + loss_psi), or both
            # optimizing distance mattrix (drmsd) and tortion angles separately probably gives better rerults. 
            # loss_all = loss_drmsd   # distance matrix loss
            # loss_all = loss_phi + loss_psi  # torsion angle loss
            
            loss_all = loss_drmsd * dist_loss_weight + (loss_phi + loss_psi) * torsion_loss_weight
            

        # Compute gradient 
        grads = tape.gradient(loss_all, model.trainable_variables)  # loss includ both distance matrix and torsion angles 
        # grads = tape.gradient(loss_all, model.trainable_variables)
        grads, global_norm = tf.clip_by_global_norm(grads, 5.0)
        
        # Backprop
        optimizer.apply_gradients(zip(grads, model.trainable_variables), global_step)

        # Record metrics
        train_rmsd_dist_norm(loss_drmsd_normalized)
        train_rmsd_dist(loss_drmsd)
        train_rmsd_angle((loss_phi + loss_psi)/2)
        train_rmsd_all(loss_all)

        if it % 10 == 0:
            print("Epoch {:04d} Batch {:03d}/{:03d}: Loss: {:.5g}, dist RMSD: {:.5g}, (length-normalized): {:.5g}, angle RMSD: {:.5g}".format(
                epoch, it, n_iter, loss_all, loss_drmsd, loss_drmsd_normalized, (loss_phi + loss_psi)/2))
        
    
    ###################################  
    # Validation 
    ################################### 
    idx_val = np.arange(X_val.shape[0])
    n_iter_val = int(len(idx_val) / batch_size) + 1  # use all

    val_rmsd_dist = tf.metrics.Mean()
    val_rmsd_dist_norm = tf.metrics.Mean()
    val_rmsd_angle = tf.metrics.Mean()
    val_rmsd_all = tf.metrics.Mean()

    for it in range(n_iter_val):
        idx_batch = idx_val[it * batch_size : (it+1) * batch_size]
        batch_seq = tf.convert_to_tensor(X_val[idx_batch])
        batch_q8s = tf.convert_to_tensor(X_val_q8s[idx_batch])
        batch_seqlen = tf.convert_to_tensor(X_val_seqlen[idx_batch], dtype=tf.float32)
        batch_dcalphas = tf.convert_to_tensor(y_val_dist_matrix[idx_batch])
        batch_msa = tf.convert_to_tensor(X_val_msa[idx_batch])

        with tf.GradientTape() as tape:
            torsion_angles, y_out = model([batch_seq, batch_q8s, batch_msa])
            phi, psi = torsion_angles[:, :, 0], torsion_angles[:, :, 1]
            phi_scaled, psi_scaled = phi * angle_scale, psi * angle_scale  # from [-pi, pi] to [-180, 180]
            loss_phi_batch = rmsd_torsion_angle(phi_scaled, y_val_phis[idx_batch], batch_seqlen)
            loss_psi_batch = rmsd_torsion_angle(psi_scaled, y_val_psis[idx_batch], batch_seqlen)
            loss_phi = tf.reduce_mean(loss_phi_batch)  
            loss_psi = tf.reduce_mean(loss_psi_batch)

            # torsion_angles are on the scale of [-3.14, 3.14], the target labels are on the scale of [-180, 180]. Need to convert after prediction
            dist_matrix, coordinates = DistanceMatrix()(torsion_angles)
            loss_drmsd_batch = drmsd_dist_matrix(dist_matrix, batch_dcalphas, batch_seqlen)
            loss_drmsd = tf.reduce_mean(loss_drmsd_batch)
            loss_drmsd_normalized = tf.reduce_mean(loss_drmsd_batch / tf.sqrt(batch_seqlen))

            #loss_all = loss_drmsd
            # loss_all = loss_phi + loss_psi
            loss_all = loss_drmsd * dist_loss_weight + (loss_phi + loss_psi) * torsion_loss_weight

        # no gradient descent during validation
        val_rmsd_dist_norm(loss_drmsd_normalized)
        val_rmsd_dist(loss_drmsd)
        val_rmsd_angle((loss_phi + loss_psi)/2)
        val_rmsd_all(loss_all)

        if it == 0:
            plot_pred = dist_matrix[:8].numpy()
            plot_gt = batch_dcalphas[:8].numpy()
            plot_names = pdb_names_val[idx_batch][:8]
            plot_len = batch_seqlen[:8].numpy().astype(int)
            plot_rmds = loss_drmsd_batch[:8].numpy()
            plot_dist_matrix(plot_pred, plot_gt, plot_names, plot_len, plot_rmds,
                os.path.join(log_dir, "plot_distance_matrix_epoch-{}.pdf".format(epoch)))

    ################################### 

    # plot history if not using tensorboard
    train_loss_history.append([train_rmsd_all.result(), train_rmsd_dist.result(), train_rmsd_dist_norm.result(), train_rmsd_angle.result()])
    val_loss_history.append([val_rmsd_all.result(), val_rmsd_dist.result(), val_rmsd_dist_norm.result(), val_rmsd_angle.result()])

    print("\n===========================================\n")
    print("Epoch {:03d}: Train Loss: {:.5g}, dist RMSD: {:.5g}, (length-normalized): {:.5g}, angle RMSD: {:.5g} \n ".format(
        epoch, *train_loss_history[-1]) + 
        "             Val Loss: {:.5g}, dist RMSD: {:.5g}, (length-normalized): {:.5g}, angle RMSD: {:.5g}".format(
        *val_loss_history[-1]))
    print("\n===========================================\n")

    # save model checkpoints (this overwrites previous checkpoints)
    tf.saved_model.save(model, log_dir)

    # plot
    train_loss_arr = np.array(train_loss_history).transpose()
    val_loss_arr = np.array(val_loss_history).transpose()
    plot_train_val(train_loss_arr[0], val_loss_arr[0], 
            title="RMSD (dist + angle) loss", savepath=os.path.join(log_dir, "rmsd_all.pdf"))
    plot_train_val(train_loss_arr[1], val_loss_arr[1], 
            title="RMSD (distance matrix) loss", savepath=os.path.join(log_dir, "rmsd_distance_matrix.pdf"))
    plot_train_val(train_loss_arr[2], val_loss_arr[2], 
            title="RMSD (distance matrix, lengh-normalized) loss", savepath=os.path.join(log_dir, "rmsd_distance_matrix_normalized.pdf"))
    plot_train_val(train_loss_arr[3], val_loss_arr[3], 
            title="RMSD (torsion angles) loss", savepath=os.path.join(log_dir, "rmsd_torsion_angles.pdf"))

# Prepare Output

In [0]:
import os, sys, time 
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import *
from model_utils import *

print(tf.__version__)

########## load data and model
data_dir = "/gdrive/My Drive/Colab Notebooks/DL-HW2/cu-deep-learning-spring19-hw2"
model_dir = "/gdrive/My Drive/Colab Notebooks/DL-HW2/logs/20190312-221750-ipykernel_launcher"  # or your model dir

data_fields = [[] for _ in range(6)]
with open(os.path.join(data_dir, 'test.pkl'), 'rb') as f:
    data = pickle.load(f)
    for j in range(len(data_fields)):
        data_fields[j] = data[j]

indices, pdbs, length_aas, pdb_aas, q8s, msas = data_fields
msas = [np.stack(x).transpose().astype(np.float32) for x in msas]

maxlen_seq = 384

# Input data 
test_input_seqs = pdb_aas
test_input_grams = seq2ngrams(test_input_seqs, n=1)
test_q8s = q8s
test_q8s_grams = seq2ngrams(test_q8s, n=1)

# Tokenizer encoder for the input sequence
with open(os.path.join(model_dir, 'tokenizer_encoder.pickle'), 'rb') as handle:
    tokenizer_encoder = pickle.load(handle)
with open(os.path.join(model_dir, 'tokenizer_encoder_q8s.pickle'), 'rb') as handle:
    tokenizer_encoder_q8s = pickle.load(handle)

# Tokenize the input sequences for use in testing
test_input_data = tokenizer_encoder.texts_to_sequences(test_input_grams)
test_input_data = sequence.pad_sequences(test_input_data, maxlen = maxlen_seq, padding = 'post', truncating='post')
test_q8s_data = tokenizer_encoder_q8s.texts_to_sequences(test_q8s_grams)
test_q8s_data = sequence.pad_sequences(test_q8s_data, maxlen = maxlen_seq, padding = 'post', truncating='post')

# extra MSA features 
msa_dim = msas[0].shape[1]
msas_padded = np.zeros([len(msas), maxlen_seq, msa_dim], dtype=np.float32)
for i in range(len(msas)):
    msas_padded[i, :msas[i].shape[0], :] = msas[i]

X_test = tf.convert_to_tensor(test_input_data, dtype=tf.float32)
X_test_q8s = tf.convert_to_tensor(test_q8s_data, dtype=tf.float32)
X_test_msa = tf.convert_to_tensor(msas_padded, dtype=tf.float32)
X_test_seqlen = length_aas



# prediction
angle_scale = 180 / np.pi 

imported = tf.saved_model.load(model_dir)
# loading a model from tf.saved_model.save() gives a forward pass function
model_func = imported.signatures['serving_default']
model_out = model_func(input_7=X_test, input_8=X_test_q8s, input_9=X_test_msa)
torsion_angles = model_out['torsion_angles_2']

phi, psi = torsion_angles[:, :, 0], torsion_angles[:, :, 1]  
phi_scaled, psi_scaled = phi * angle_scale, psi * angle_scale  # from [-pi, pi] to [-180, 180]
dist_matrix, coordinates = DistanceMatrix()(torsion_angles)

phi_scaled, psi_scaled = phi_scaled.numpy(), psi_scaled.numpy()
dist_matrix = dist_matrix.numpy()

phi_output, psi_output, dist_matrix_output = [], [], []
for i in range(X_test.shape[0]):
    phi_output.append(phi_scaled[i, :X_test_seqlen[i]])
    psi_output.append(psi_scaled[i, :X_test_seqlen[i]])
    dist_matrix_output.append(dist_matrix[i, :X_test_seqlen[i], :X_test_seqlen[i]])

output = [dist_matrix_output, psi_output, phi_output]
with open(os.path.join(model_dir, 'predictions.pkl'), 'wb') as handle:
    pickle.dump(output, handle)



2.0.0-dev20190227


In [0]:
import pickle, sys, os
import pandas as pd
import numpy as np

# load data
model_dir = "/gdrive/My Drive/Colab Notebooks/DL-HW2/logs/20190312-221750-ipykernel_launcher"  # or your model dir 
datafile = model_dir +"/predictions.pkl"
with open(datafile, "rb") as f:
    data = pickle.load(f)
dist, psi, phi = data

# load protein names
testfile = "/gdrive/My Drive/Colab Notebooks/DL-HW2/cu-deep-learning-spring19-hw2/test.csv"
test_input = pd.read_csv(testfile, header=None)
protein_names = np.array(test_input.iloc[:,1])
protein_len = np.array(test_input.iloc[:,2])

# concatenate all output to one-dimensional
all_data = []
all_names = []
for i, pname in enumerate(protein_names):
    dist_flat = dist[i].ravel()
    array = np.concatenate([dist_flat, psi[i], phi[i]])
    all_data.append(array)

    length = protein_len[i]
    dist_names = ["{}_d_{}_{}".format(pname, i + 1, j + 1) for i in range(length) for
            j in range(length)]

    psi_names = ["{}_psi_{}".format(pname, i + 1) for i in range(length)]
    phi_names = ["{}_phi_{}".format(pname, i + 1) for i in range(length)]
    row_names = np.array(dist_names + psi_names + phi_names)
    all_names.append(row_names)

all_data = np.concatenate(all_data)
all_names = np.concatenate(all_names)
output = {"Id": all_names, "Predicted": all_data}
output = pd.DataFrame(output)
output.to_csv(os.path.join(model_dir, "submission.csv"), index=False)
